# Importando o Spark

In [ ]:
!pip install pySpark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master('local[*]').appName("Challenge Alura Spark").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
spark

# Montando no Drive para caso for usar no Colab





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importando os Dados, colocando em um DataFrame

In [ ]:
dados = spark.read.json('/content/drive/MyDrive/Challenge_Pyspark_Alura/Data/dataset_bruto.json')

In [ ]:
linhas = dados.count()
colunas = len(dados.columns)

In [ ]:
print(f'A quantidade de linhas é {linhas} e a quantidade de colunas é {colunas}')

A quantidade de linhas é 89083 e a quantidade de colunas é 3


In [ ]:
# Estrutura da Base de Dados

dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [ ]:
dados.show()

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
|{0, [20], [15], [...|[{2de09d46-dc0d-4...|{3c7057f5-0923-42...|
|{3, [43], [43], [...|[{147a80d9-cd40-4...|{5a9736b5-aaa0-4a...|
|{2, [42], [42], [...|[{35740004-063d-4...|{ec48d96a-137c-49...|
|{0, [], [12], [0]...|[{6d3d2aec-c96f-4...|{dad7db63-e19c-44...|
|{1, [41], [41], [...|[{3d404069-418e-4...|{a845f35f-3ab3-46...|
|{3, [43], [43], [...|[{5b8a8de7-2088-4...|{8c8de313-45d5-4c...|
|{0, [10], [4], [0...|[{5385da42-8276-4...|{0b1d4ee0-2bc3-4c...|
|{0, [12], [12], [...|[{7

# Separando os Dados das Colunas e deixando apenas a coluna Anuncio (Porque as outras não são necessárias)

In [ ]:
anuncio = dados.select('anuncio.*')

In [ ]:
anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

# Filtrar a base de dados (tipo_uso, tipo_unidade e tipo_anuncio)

In [ ]:
anuncio.groupby(['tipo_uso', 'tipo_unidade', 'tipo_anuncio']).count().orderBy('count', ascending=False).show()

+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio|count|
+-----------+------------+------------+-----+
|Residencial| Apartamento|       Usado|66562|
|Residencial|        Casa|       Usado|10224|
|Residencial|      Outros|       Usado| 7502|
|  Comercial|      Outros|       Usado| 4443|
|Residencial| Apartamento|  Lançamento|  235|
|  Comercial|        Casa|       Usado|   92|
|Residencial|      Outros|  Lançamento|   15|
|  Comercial| Apartamento|       Usado|    4|
|  Comercial|      Outros|  Lançamento|    3|
|Residencial|        Casa|  Lançamento|    3|
+-----------+------------+------------+-----+



## Coluna: Tipo_Uso

In [ ]:
anuncio\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



In [ ]:
anuncio=anuncio.filter('tipo_uso="Residencial"')

In [ ]:
anuncio\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|84541|
+-----------+-----+



## Coluna: TIpo_Unidade

In [ ]:
anuncio\
    .select('tipo_unidade')\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros| 7517|
| Apartamento|66797|
|        Casa|10227|
+------------+-----+



In [ ]:
anuncio = anuncio.filter('tipo_unidade=="Apartamento"')

In [ ]:
anuncio\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66797|
+------------+-----+



## Coluna: TIpo_Anuncio

In [ ]:
anuncio\
    .select('tipo_anuncio')\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
|  Lançamento|  235|
+------------+-----+



In [ ]:
anuncio = anuncio.filter('tipo_anuncio=="Usado"')

In [ ]:
anuncio\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



## Frequência que aparece o anuncio que seja uso Residencial, do tipo Apartamento e usado.

In [ ]:
freq = anuncio\
  .select('*')\
  .where('tipo_uso=="Residencial"')\
  .where('tipo_unidade=="Apartamento"')\
  .where('tipo_anuncio=="Usado"')

In [ ]:
freq.count()

66562

# Transformando as colunas dos cômodos dos imóveis de listas para inteiros (quartos, suites, banheiros, vaga, area_total e area_util)

In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType,IntegerType

In [ ]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [ ]:
listas_int = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

In [ ]:
for col in listas_int:
    anuncio\
    .select(f.size(f.col(col)).alias((col)))\
    .groupby(col)\
    .count()\
    .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+



In [ ]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [ ]:
anuncio = anuncio.select([f.col(c)[0].alias(c) if c in listas_int else c for c in anuncio.columns])

# Deixando apenas as colunas Bairro e Zona no DataFrame 

In [ ]:
anuncio\
    .select('endereco.*')\
    .show()

+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|              bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|
+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.919851|-43.634034|  BR|Estrada de Santa ...|Zona Oeste|
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.928108|-43.635375|  BR|Estrada de Santa ...|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948756|-43.582824|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|Rio de Janeiro|Rio de Janeiro|-22.888194|-43.629602|  BR|Estrada da Paciência|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948291|-43.582205|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|R

In [ ]:
anuncio\
    .select('endereco.bairro', 'endereco.zona')\
    .show()

+--------------------+----------+
|              bairro|      zona|
+--------------------+----------+
|           Paciência|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|  Pedra de Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|        Campo Grande|Zona Oeste|
|          Santa Cruz|Zona Oeste|
|           Guaratiba|Zona Oeste|
|Recreio dos Bande...|Zona Oeste|
|              Cosmos|Zona Oeste|
|            Realengo|Zona Oeste|
|              Cosmos|Zona Oeste|
|     Todos os Santos|Zona Norte|
+--------------------+----------+
only showing top 20 rows



In [ ]:
anuncio = anuncio\
    .select('*',  'endereco.bairro', 'endereco.zona')\
    .drop('endereco')
anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|           Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|           Paciência|Zona Oeste|
|    1|        41|       41|        1|[P

# Transformando cada campo da coluna "valores" em uma coluna separada e deixando apenas do Tipo Venda

In [ ]:
anuncio\
    .select('valores')\
    .show()

+--------------------+
|             valores|
+--------------------+
|[{245, null, Vend...|
|[{0, 0, Venda, 15...|
|[{0, 0, Venda, 20...|
|[{285, null, Vend...|
|[{245, null, Vend...|
|[{285, null, Vend...|
|[{250, null, Vend...|
|[{245, null, Vend...|
|[{245, null, Vend...|
|[{240, null, Vend...|
|[{0, 0, Venda, 15...|
|[{240, 0, Venda, ...|
|[{245, null, Vend...|
|[{290, null, Vend...|
|[{285, null, Vend...|
|[{0, 0, Venda, 30...|
|[{null, null, Ven...|
|[{280, 0, Venda, ...|
|[{280, null, Vend...|
|[{0, 0, Venda, 26...|
+--------------------+
only showing top 20 rows



In [ ]:
anuncio\
      .select(f.explode('valores').alias('valores'))\
      .select('valores.*')\
      .filter('valores.tipo=="Venda"')\
      .count()

66562

In [ ]:
anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .show(50, truncate=False)

+------------------------------------+----------+----+-------+-----+
|id                                  |condominio|iptu|tipo   |valor|
+------------------------------------+----------+----+-------+-----+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|245       |null|Venda  |15000|
|085bab2c-87ad-452a-bd0f-8b1451509f84|0         |0   |Venda  |15000|
|18d22cbe-1b86-4764-8def-70c615f15a61|0         |0   |Venda  |20000|
|bed8a354-9317-4426-b27e-1c798f864271|285       |null|Venda  |20000|
|12a13315-d67f-48f0-9497-017b83252781|245       |null|Venda  |15000|
|a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e|285       |null|Venda  |20000|
|a6e3173b-c950-4db4-9b7b-80c44bd75e90|250       |null|Venda  |15000|
|2e6e5dfb-206c-4968-944b-ea4c3918b50d|245       |null|Venda  |15000|
|99f8d0f9-95a4-4613-a55d-c949e7a73e90|245       |null|Venda  |15000|
|b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8|240       |null|Venda  |17999|
|dc99d9e1-4c63-41f7-a53d-678b2b3ccb52|0         |0   |Venda  |15000|
|aa3606d8-8bcd-45ec-a609-d68a58a12

In [ ]:
anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .filter('valores.tipo=="Venda"')\
    .show(truncate=False)

+------------------------------------+----------+----+-----+-----+
|id                                  |condominio|iptu|tipo |valor|
+------------------------------------+----------+----+-----+-----+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|245       |null|Venda|15000|
|085bab2c-87ad-452a-bd0f-8b1451509f84|0         |0   |Venda|15000|
|18d22cbe-1b86-4764-8def-70c615f15a61|0         |0   |Venda|20000|
|bed8a354-9317-4426-b27e-1c798f864271|285       |null|Venda|20000|
|12a13315-d67f-48f0-9497-017b83252781|245       |null|Venda|15000|
|a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e|285       |null|Venda|20000|
|a6e3173b-c950-4db4-9b7b-80c44bd75e90|250       |null|Venda|15000|
|2e6e5dfb-206c-4968-944b-ea4c3918b50d|245       |null|Venda|15000|
|99f8d0f9-95a4-4613-a55d-c949e7a73e90|245       |null|Venda|15000|
|b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8|240       |null|Venda|17999|
|dc99d9e1-4c63-41f7-a53d-678b2b3ccb52|0         |0   |Venda|15000|
|aa3606d8-8bcd-45ec-a609-d68a58a12014|240       |0   |Venda|19

In [ ]:
valores = anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .filter('valores.tipo=="Venda"')

# Unir os dois DataFrames usando o 'id' como primary key

In [ ]:
anuncio.join(valores, 'id', how='inner').count()

66562

In [ ]:
anuncio.join(valores, 'id', how='inner').show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|[{285, null, Vend...|            Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado|

In [ ]:
anuncio.join(valores, 'id', how='inner').drop('valores').show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|               Irajá|  Zona Norte|       170|   0|Vend

In [ ]:
anuncio = anuncio.join(valores, 'id', how='inner').drop('valores')

# Salvando os dados no formato parquet 

In [ ]:
anuncio.write.parquet(
    path='/content/drive/MyDrive/Challenge_Pyspark_Alura/parquet',
    mode='overwrite'
)

In [ ]:
anuncio_Parquet = spark.read.parquet('/content/drive/MyDrive/Challenge_Pyspark_Alura/parquet/part-00000-67f0ea53-49e1-482e-954e-8c385eabc181-c000.snappy.parquet')

In [ ]:
anuncio_Parquet.count()

66562

# Salvando os dados em CSV

In [ ]:
from pyspark.sql.types import StringType

In [ ]:
anuncio= anuncio.withColumn('caracteristicas', anuncio['caracteristicas'].cast(StringType()))
anuncio.select('caracteristicas').printSchema()

root
 |-- caracteristicas: string (nullable = true)



In [ ]:
anuncio = anuncio.drop('endereco')
anuncio = anuncio.drop('valores')

In [ ]:
anuncio.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [ ]:
anuncio.write.csv(
    path='/content/drive/MyDrive/Challenge_Pyspark_Alura/csv',
    mode='overwrite',
    sep=';',
    header=True,
)

# Comparando o Desempenho de Leitura

In [ ]:
%time

parquet = spark.read.parquet('/content/drive/MyDrive/Challenge_Pyspark_Alura/parquet')
parquet.show()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.78 µs
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residen

In [ ]:
%time

csv = spark.read.csv('/content/drive/MyDrive/Challenge_Pyspark_Alura/csv')
csv

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


DataFrame[_c0: string]

In [ ]:
spark.stop()